In [2]:
!pip install -q sentence-transformers chromadb gradio

In [4]:
# Eğer extracted_texts varsa birleştir; yoksa elle kısa belgeler kullan.
try:
    documents = extracted_texts  # hücre 2 çalıştıysa burada veri olur
    if not documents:
        raise Exception("no extracted texts")
except:
    documents = [
        {"id":"info1", "text":"Sınav tarihi: 23.10.2025 Perşembe. Başlangıç saati 10:00, bitiş 12:40. Toplam süre 2 saat 40 dakika."},
        {"id":"info2", "text":"Sınav bölümleri: Writing, Listening, Reading."},
        {"id":"info3", "text":"Sorumlu konular: İlk 3 ünite. Language Hub kitap etkinliklerine benzer sorular çıkabilir."},
        {"id":"faq1", "text":"Soru: Sınav ne zaman? Cevap: 23.10.2025, 10:00'da başlıyor."},
        {"id":"faq2", "text":"Soru: Sınav süresi ne kadar? Cevap: 2 saat 40 dakika (10:00 - 12:40)."}
    ]

print("Toplam doküman sayısı:", len(documents))
for d in documents[:5]:
    print("-", d["id"], "->", d["text"][:100].replace("\n"," "))

Toplam doküman sayısı: 5
- info1 -> Sınav tarihi: 23.10.2025 Perşembe. Başlangıç saati 10:00, bitiş 12:40. Toplam süre 2 saat 40 dakika.
- info2 -> Sınav bölümleri: Writing, Listening, Reading.
- info3 -> Sorumlu konular: İlk 3 ünite. Language Hub kitap etkinliklerine benzer sorular çıkabilir.
- faq1 -> Soru: Sınav ne zaman? Cevap: 23.10.2025, 10:00'da başlıyor.
- faq2 -> Soru: Sınav süresi ne kadar? Cevap: 2 saat 40 dakika (10:00 - 12:40).


In [5]:
from sentence_transformers import SentenceTransformer
model_name = "all-MiniLM-L6-v2"  # hafif, hızlı
embed_model = SentenceTransformer(model_name)
print("Embedding modeli yüklendi:", model_name)

def embed_texts(texts):
    # texts: list of strings
    return embed_model.encode(texts, show_progress_bar=False, convert_to_numpy=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding modeli yüklendi: all-MiniLM-L6-v2


In [6]:
import chromadb
from chromadb.config import Settings

# Local/ in-memory Chroma client
client = chromadb.Client(Settings(anonymized_telemetry=False))

# Eğer aynı isimde koleksiyon varsa önce silip yeniden oluştur (temizlik)
col_name = "exam_info_collection"
try:
    if col_name in [c.name for c in client.list_collections()]:
        client.delete_collection(col_name)
except Exception:
    pass

collection = client.create_collection(name=col_name)

# Prepare lists
ids = [d["id"] for d in documents]
texts = [d["text"] for d in documents]
metadatas = [{"source": d.get("id","local")} for d in documents]

# compute embeddings once (use embed_texts)
embs = embed_texts(texts)

# add to collection with embeddings
collection.add(ids=ids, documents=texts, metadatas=metadatas, embeddings=embs.tolist())
print("✅ Chroma'ya eklendi:", collection.count(), "doküman")

✅ Chroma'ya eklendi: 5 doküman


In [7]:
def retrieve(query, k=3):
    # chroma query; n_results = k
    res = collection.query(query_texts=[query], n_results=k, include=["documents","metadatas","distances"])
    docs = res["documents"][0]
    metas = res["metadatas"][0]
    distances = res["distances"][0]
    hits = []
    for d, m, dist in zip(docs, metas, distances):
        hits.append({"text": d, "metadata": m, "distance": dist})
    return hits

# Test
q = "Sınav ne zaman başlıyor?"
hits = retrieve(q, k=3)
for i,h in enumerate(hits,1):
    print(f"#{i} (dist={h['distance']:.4f}) -> {h['text'][:120].replace(chr(10),' ')}")

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:07<00:00, 10.6MiB/s]


#1 (dist=0.3581) -> Soru: Sınav ne zaman? Cevap: 23.10.2025, 10:00'da başlıyor.
#2 (dist=0.6189) -> Sınav tarihi: 23.10.2025 Perşembe. Başlangıç saati 10:00, bitiş 12:40. Toplam süre 2 saat 40 dakika.
#3 (dist=0.6940) -> Soru: Sınav süresi ne kadar? Cevap: 2 saat 40 dakika (10:00 - 12:40).


In [8]:
def generate_answer_rule_based(question, k=3):
    hits = retrieve(question, k=k)
    if not hits:
        return "Bu konuda veri yok."
    # En yakın dokümanı al ve kısaltarak cevap dön
    top = hits[0]
    answer = f"Bulunan bilgi (kaynak: {top['metadata'].get('source','bilgi')}):\n{top['text']}"
    return answer

# Test
print(generate_answer_rule_based("Sınav süresi kaç dakika?"))

Bulunan bilgi (kaynak: faq2):
Soru: Sınav süresi ne kadar? Cevap: 2 saat 40 dakika (10:00 - 12:40).


In [9]:
import gradio as gr

def chat_fn(user_input):
    return generate_answer_rule_based(user_input, k=2)

iface = gr.Interface(fn=chat_fn,
                     inputs=gr.Textbox(lines=2, placeholder="Sınav hakkında bir soru yaz..."),
                     outputs="text",
                     title="Hazırlık Sınavı - Mini Chatbot")
# Colab için share=True kullan; çalıştırınca bir URL verecek
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2b1cfb17dccf7dbbef.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
# birkaç örnek soru dene
for q in ["Sınav ne zaman?", "Sınav süresi nedir?", "Hangi bölümler var?"]:
    print("Soru:", q)
    print(generate_answer_rule_based(q))
    print("-"*40)

Soru: Sınav ne zaman?
Bulunan bilgi (kaynak: faq1):
Soru: Sınav ne zaman? Cevap: 23.10.2025, 10:00'da başlıyor.
----------------------------------------
Soru: Sınav süresi nedir?
Bulunan bilgi (kaynak: faq2):
Soru: Sınav süresi ne kadar? Cevap: 2 saat 40 dakika (10:00 - 12:40).
----------------------------------------
Soru: Hangi bölümler var?
Bulunan bilgi (kaynak: info2):
Sınav bölümleri: Writing, Listening, Reading.
----------------------------------------


In [11]:
# FL3 seviyesiyle ilgili bilgi dokümanı ekleme
new_docs2 = [
    {"id": "info_fl3", "text": "Bu sınav FL3 kurunda olan öğrenciler için hazırlanmıştır. Yani sınav içeriği ve zorluk seviyesi FL3 seviyesine uygundur."},
    {"id": "faq_fl3", "text": "Soru: Bu sınav hangi kur için? Cevap: FL3 kurundaki öğrenciler içindir."}
]

# Embedding oluştur
new_texts2 = [d["text"] for d in new_docs2]
new_ids2 = [d["id"] for d in new_docs2]
new_metas2 = [{"source": d["id"]} for d in new_docs2]
new_embs2 = embed_texts(new_texts2)

# Koleksiyona ekle
collection.add(ids=new_ids2, documents=new_texts2, metadatas=new_metas2, embeddings=new_embs2.tolist())

print("✅ FL3 bilgisi eklendi! Şu anda toplam belge sayısı:", collection.count())

✅ FL3 bilgisi eklendi! Şu anda toplam belge sayısı: 7


In [12]:
iface = gr.Interface(
    fn=chat_fn,
    inputs=gr.Textbox(lines=2, placeholder="Sınav hakkında soru yaz..."),
    outputs="text",
    title="Hazırlık Sınavı Chatbot (FL3 Kuru)",
    description="💬 Bu chatbot FL3 kurundaki öğrenciler için hazırlanmıştır. Sınav tarihi, saat, içerik ve konular hakkında bilgi verebilir."
)
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ccd9184a6caf2ed4f2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [13]:
# Sınav soru tipleriyle ilgili yeni dokümanlar
new_docs3 = [
    {
        "id": "info_question_types",
        "text": "Sınav soru tipleri: çoktan seçmeli, boşluk doldurma ve eşleştirme sorularından oluşur. "
                "Bazı bölümlerde kısa yazma veya kelime tamamlama da bulunabilir."
    },
    {
        "id": "faq_question_types",
        "text": "Soru: Sınavda hangi tür sorular var? "
                "Cevap: Sınav çoktan seçmeli, boşluk doldurma ve eşleştirme sorularından oluşmaktadır."
    }
]

# Embedding oluştur
new_texts3 = [d["text"] for d in new_docs3]
new_ids3 = [d["id"] for d in new_docs3]
new_metas3 = [{"source": d["id"]} for d in new_docs3]
new_embs3 = embed_texts(new_texts3)

# Koleksiyona ekle
collection.add(ids=new_ids3, documents=new_texts3, metadatas=new_metas3, embeddings=new_embs3.tolist())

print("✅ Soru tipi bilgisi eklendi! Şu anda toplam belge sayısı:", collection.count())

✅ Soru tipi bilgisi eklendi! Şu anda toplam belge sayısı: 9


In [14]:
# Sınav yeri bilgisi ekleme
new_docs4 = [
    {
        "id": "info_location",
        "text": "Sınav yeri: Hacettepe Üniversitesi Beytepe Kampüsü Yabancı Diller Yüksekokulu (YDYO) binasında gerçekleşecektir."
    },
    {
        "id": "faq_location",
        "text": "Soru: Sınav nerede yapılacak? "
                "Cevap: Hacettepe Üniversitesi Beytepe Kampüsü YDYO binasında yapılacaktır."
    }
]

# Embedding oluştur
new_texts4 = [d["text"] for d in new_docs4]
new_ids4 = [d["id"] for d in new_docs4]
new_metas4 = [{"source": d["id"]} for d in new_docs4]
new_embs4 = embed_texts(new_texts4)

# Koleksiyona ekle
collection.add(ids=new_ids4, documents=new_texts4, metadatas=new_metas4, embeddings=new_embs4.tolist())

print("✅ Sınav yeri bilgisi eklendi! Şu anda toplam belge sayısı:", collection.count())

✅ Sınav yeri bilgisi eklendi! Şu anda toplam belge sayısı: 11


In [15]:
# Sınava hazırlanma önerileri (kaynak kitaplar) bilgisi
new_docs5 = [
    {
        "id": "info_study_materials",
        "text": "Sınava hazırlanmak isteyen öğrenciler Language Hub kitabındaki üniteleri tekrar etmeli "
                "ve Writer 2 kitabındaki örnek sorulara göz atmalıdır. "
                "Bu kaynaklardaki etkinlikler sınav formatına benzerdir."
    },
    {
        "id": "faq_study_materials",
        "text": "Soru: Sınava nasıl hazırlanabilirim? "
                "Cevap: Language Hub kitabına ve Writer 2 kitabındaki örnek sorulara çalışabilirsiniz. "
                "Bu kitaplarda yer alan etkinlikler sınav sorularına benzer yapıdadır."
    }
]

# Embedding oluştur
new_texts5 = [d["text"] for d in new_docs5]
new_ids5 = [d["id"] for d in new_docs5]
new_metas5 = [{"source": d["id"]} for d in new_docs5]
new_embs5 = embed_texts(new_texts5)

# Koleksiyona ekle
collection.add(ids=new_ids5, documents=new_texts5, metadatas=new_metas5, embeddings=new_embs5.tolist())

print("✅ Sınava hazırlık bilgisi eklendi! Şu anda toplam belge sayısı:", collection.count())

✅ Sınava hazırlık bilgisi eklendi! Şu anda toplam belge sayısı: 13
